In [2]:
import pandas as pd
from datetime import date, timedelta
import numpy as np

In [3]:
df_cases_daily = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-data.csv")
df_cases_agg = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-table-data.csv")
df_vac = pd.read_csv("https://covid19.who.int/who-data/vaccination-data.csv")
df_vac_meta = pd.read_csv("https://covid19.who.int/who-data/vaccination-metadata.csv")
df_public_health = pd.read_csv("https://covid19.who.int/who-data/phsm-severity-data.csv")

In [4]:
for df in [df_cases_daily, df_cases_agg, df_vac, df_vac_meta, df_public_health]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144570 entries, 0 to 144569
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      144570 non-null  object
 1   Country_code       143960 non-null  object
 2   Country            144570 non-null  object
 3   WHO_region         144570 non-null  object
 4   New_cases          144570 non-null  int64 
 5   Cumulative_cases   144570 non-null  int64 
 6   New_deaths         144570 non-null  int64 
 7   Cumulative_deaths  144570 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 8.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, Global to Tuvalu
Data columns (total 12 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Name                                                          237

In [5]:
today = date.today()
yesterday = today - timedelta(days=1)
daysAgo7 = today - timedelta(days=7)
yesterday_str = yesterday.strftime("%Y-%m-%d")
daysAgo7_str = daysAgo7.strftime("%Y-%m-%d")

In [6]:
df_7daysAgo = df_cases_daily[df_cases_daily["Date_reported"] >= daysAgo7_str]
df_details_agg = df_7daysAgo.groupby(["Country_code", "Country", "WHO_region"]).agg(
    date = pd.NamedAgg(column="Date_reported", aggfunc=max),
    cases7days = pd.NamedAgg(column="New_cases", aggfunc=sum),
    deaths7days = pd.NamedAgg(column="New_deaths", aggfunc=sum)
)
df_details_agg.reset_index(inplace=True)
df_details_agg = df_details_agg[["Country","date","cases7days","deaths7days"]]
df_details_agg

,Country,date,cases7days,deaths7days
0,Other,2021-09-03,0,0
1,Andorra,2021-09-03,27,0
2,United Arab Emirates,2021-09-03,5934,8
3,Afghanistan,2021-09-03,368,24
4,Antigua and Barbuda,2021-09-03,144,1
...,...,...,...,...
231,Yemen,2021-09-03,240,39
232,Mayotte,2021-09-03,136,0
233,South Africa,2021-09-03,49387,1727
234,Zambia,2021-09-03,1001,21


In [7]:
df_vac["pop"] = 100*df_vac["TOTAL_VACCINATIONS"]/df_vac["TOTAL_VACCINATIONS_PER100"]
df_vac_red = df_vac[["COUNTRY", "PERSONS_VACCINATED_1PLUS_DOSE_PER100", "PERSONS_FULLY_VACCINATED_PER100","pop"]]
df_vac_red.rename(columns={"COUNTRY": "Country", "PERSONS_VACCINATED_1PLUS_DOSE_PER100": "Perc1Vacc", "PERSONS_FULLY_VACCINATED_PER100": "PercFullyVacc"}, inplace=True)
df_vac_red

C:\Users\nikol\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Country,Perc1Vacc,PercFullyVacc,pop
0,Afghanistan,1.986,1.134,3.893121e+07
1,Albania,28.639,22.026,2.877821e+06
2,Algeria,7.802,1.653,4.385078e+07
3,American Samoa,54.338,45.251,5.519721e+04
4,Andorra,66.064,52.567,7.726480e+04
...,...,...,...,...
222,Viet Nam,17.062,2.296,9.733962e+07
223,Wallis and Futuna,44.033,43.118,1.124600e+04
224,Yemen,1.000,0.045,2.983554e+07
225,Zambia,1.653,1.086,1.838347e+07


In [8]:
df_vac

,COUNTRY,ISO3,WHO_REGION,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,PERSONS_VACCINATED_1PLUS_DOSE,TOTAL_VACCINATIONS_PER100,PERSONS_VACCINATED_1PLUS_DOSE_PER100,PERSONS_FULLY_VACCINATED,PERSONS_FULLY_VACCINATED_PER100,VACCINES_USED,FIRST_VACCINE_DATE,NUMBER_VACCINES_TYPES_USED,pop
0,Afghanistan,AFG,EMRO,REPORTING,2021-08-30,1979652,773002.0,5.085,1.986,441371.0,1.134,"Beijing CNBG - BBIBP-CorV,Janssen - Ad26.COV 2...",2021-02-22,4.0,3.893121e+07
1,Albania,ALB,EURO,OWID,2021-08-30,1458048,824170.0,50.665,28.639,633878.0,22.026,"AstraZeneca - AZD1222, Pfizer BioNTech - Comir...",NaN,4.0,2.877821e+06
2,Algeria,DZA,AFRO,REPORTING,2021-08-23,4146091,3421279.0,9.455,7.802,724812.0,1.653,"Beijing CNBG - BBIBP-CorV,Gamaleya - Sputnik V...",2021-01-30,3.0,4.385078e+07
3,American Samoa,ASM,WPRO,REPORTING,2021-08-20,54425,29993.0,98.601,54.338,24977.0,45.251,"Janssen - Ad26.COV 2.5,Moderna - mRNA-1273,Pfi...",2020-12-21,3.0,5.519721e+04
4,Andorra,AND,EURO,OWID,2021-08-15,91660,51044.0,118.631,66.064,40616.0,52.567,"AstraZeneca - AZD1222, Pfizer BioNTech - Comir...",NaN,2.0,7.726480e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Viet Nam,VNM,WPRO,REPORTING,2021-08-27,18843004,16607991.0,19.358,17.062,2235013.0,2.296,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-03-08,5.0,9.733962e+07
223,Wallis and Futuna,WLF,WPRO,REPORTING,2021-07-22,9801,4952.0,87.151,44.033,4849.0,43.118,Moderna - mRNA-1273,2021-03-19,1.0,1.124600e+04
224,Yemen,YEM,EMRO,REPORTING,2021-07-27,311483,298161.0,1.044,1.000,13322.0,0.045,SII - Covishield,2021-04-20,1.0,2.983554e+07
225,Zambia,ZMB,AFRO,REPORTING,2021-08-23,503707,303966.0,2.740,1.653,199741.0,1.086,SII - Covishield,2021-04-14,1.0,1.838347e+07


In [31]:
pd.set_option('display.max_rows', 500)

result = pd.merge(df_details_agg, df_vac_red, how="inner", on=["Country"])
df_public_health_adj = df_public_health.sort_values(by=["DATE_START"], ascending=False)
df_public_health_adj = df_public_health_adj.drop_duplicates(subset=["COUNTRY"], keep='first')
df_public_health_adj.reset_index(inplace=True, drop=True)
df_public_health_ren = df_public_health_adj[["COUNTRY","TRAVEL"]].rename(columns={"COUNTRY": "Country"}, inplace=False)
result = pd.merge(result, df_public_health_ren, how="inner", on=["Country"])
result["cases7daysPer100k"] = 100000*result["cases7days"]/result["pop"]
result["deaths7daysPer100k"] = 100000*result["deaths7days"]/result["pop"]
result["KPI"] = 1/(np.maximum(result["deaths7daysPer100k"], 1) * np.maximum(result["deaths7daysPer100k"], 1) * np.maximum(result["TRAVEL"], 1))*result["PercFullyVacc"]*result["Perc1Vacc"]
result.sort_values(by="KPI", ascending=False) 

,Country,date,cases7days,deaths7days,Perc1Vacc,PercFullyVacc,pop,GLOBAL_INDEX,cases7daysPer100k,deaths7daysPer100k,KPI
145,Pitcairn Islands,2021-09-03,0,0,74.000,72.000,5.000000e+01,0,0.000000,0.000000,5328.000000
148,Palau,2021-09-03,0,0,87.447,77.990,1.809199e+04,15,0.000000,0.000000,454.666102
134,Nauru,2021-09-03,0,0,70.260,66.254,1.083397e+04,11,0.000000,0.000000,423.182367
8,American Samoa,2021-09-03,0,0,54.338,45.251,5.519721e+04,7,0.000000,0.000000,351.264120
135,Niue,2021-09-03,0,0,74.289,71.075,1.617996e+03,17,0.000000,0.000000,310.593569
161,Saint Helena,2021-09-03,0,0,71.833,58.162,6.071003e+03,21,0.000000,0.000000,198.950045
52,Finland,2021-09-03,3205,6,72.800,50.760,5.525284e+06,20,58.006067,0.108592,184.766400
192,Bonaire,2021-09-03,8,0,73.980,64.571,2.091504e+04,26,38.249983,0.000000,183.729330
85,Jersey,2021-09-03,126,1,70.374,66.592,1.077962e+05,26,116.887289,0.927677,180.244054
3,Anguilla,2021-09-03,62,0,63.278,59.759,1.500199e+04,21,413.278470,0.000000,180.068095


In [32]:

df_public_health_adj[df_public_health_adj["COUNTRY"] == "Australia"]


,DATE_START,COUNTRY,ISO3,WHO_REGION,MASKS,TRAVEL,GATHERINGS,SCHOOLS,BUSINESSES,MOVEMENTS,GLOBAL_INDEX,MEASURES_IN_PLACE
153,2021-08-31,Australia,AUS,WPRO,13,100,5,80,80,20,50,Mask wearing; International travel restriction...


,Country,date,cases7days,deaths7days,Perc1Vacc,PercFullyVacc,pop,KPI
72,Gibraltar,2021-09-03,44,1,117.625,116.426,3.369095e+04,13694.608250
163,Palau,2021-09-03,0,0,87.447,77.990,1.809199e+04,6819.991530
108,Cayman Islands,2021-09-03,20,0,77.868,74.843,6.571979e+04,5827.874724
90,Isle of Man,2021-09-03,186,0,76.581,73.468,8.503212e+04,5626.252908
94,Iceland,2021-09-03,398,1,76.720,72.417,3.641339e+05,5555.832240
...,...,...,...,...,...,...,...,...
123,Republic of Moldova,2021-09-03,2301,24,NaN,17.131,4.033932e+06,NaN
125,Madagascar,2021-09-03,16,1,0.711,NaN,2.770759e+07,NaN
195,Tokelau,2021-09-03,0,0,71.704,NaN,1.349994e+03,NaN
208,Uzbekistan,2021-09-03,4528,35,28.357,NaN,3.346956e+07,NaN
